# Tutorial bookkeeping

In this notebook, we explain how to use the LaCE bookkeeping

## Read data

In [1]:
from lace.archive.pnd_archive import archivePND

#### Read full post-processing, and then training data

Cabayol23

In [6]:
archive = archivePND(postproc='Cabayol23')
# (hypercube + special pairs) * nphases * naxes * nscalings * nsnaps
nelem = (30 + 7) * 2 * 3 * 5 * 11
print(len(archive.data), nelem)

archive_training = archive.get_training_data()
# hypercube * nphases * naxes * nscalings * nsnaps
n_av_phases = 30 * 1 * 3 * 5 * 11
n_av_axes   = 30 * 2 * 1 * 5 * 11
n_av_all    = 30 * 1 * 1 * 5 * 11
print(len(archive_training), n_av_phases + n_av_axes + n_av_all)

12210 12210
9900 9900


Pedersen21

In [5]:
archive = archivePND(postproc='Pedersen21')
# (hypercube + special pairs) * nphases * naxes * nscalings * nsnaps
# some special pairs have only 1 scaling
nelem = (30 + 1) * 2 * 1 * 3 * 11 + 6 * 2 * 1 * 1 * 11
print(len(archive.data), nelem)

archive_training = archive.get_training_data()
# hypercube * nsnaps
nelem = 30 * 11
print(len(archive_training), nelem)

2178 2178
330 330


#### Leave-one-out dropping of sims (for first 3 sims)

Cabayol23

In [7]:
archive = archivePND(postproc='Cabayol23')

In [10]:
for ii in range(3):
    archive_training = archive.get_training_data(drop_sim='mpg_'+str(ii))
    # hypercube * nphases * naxes * nscalings * nsnaps
    n_av_phases = (30-1) * 1 * 3 * 5 * 11
    n_av_axes   = (30-1) * 2 * 1 * 5 * 11
    n_av_all    = (30-1) * 1 * 1 * 5 * 11
    print(len(archive_training), n_av_phases + n_av_axes + n_av_all)
    
    archive_testing = archive.get_testing_data('mpg_'+str(ii))
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem)

9570 9570
11 11


Pedersen21

In [11]:
archive = archivePND(postproc='Pedersen21')

In [13]:
for ii in range(3):
    archive_training = archive.get_training_data(drop_sim='mpg_'+str(ii))
    # hypercube * nphases * naxes * nscalings * nsnaps
    nelem = (30-1) * 1 * 1 * 1 * 11
    print(len(archive_training), nelem)
    
    archive_testing = archive.get_testing_data('mpg_'+str(ii))
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem)
    

319 319
11 11
319 319
11 11
319 319
11 11


#### Read special simulations for testing

Cabayol23

In [17]:
archive = archivePND(postproc='Cabayol23')

In [18]:
for sim in archive.list_sim_test:
    archive_testing = archive.get_testing_data(sim)
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem, archive_testing[0]['sim_label'])

11 11 mpg_central
11 11 mpg_seed
11 11 mpg_growth
11 11 mpg_neutrinos
11 11 mpg_curved
11 11 mpg_running
11 11 mpg_reio


Pedersen21

In [19]:
archive = archivePND(postproc='Pedersen21')

In [20]:
for sim in archive.list_sim_test:
    archive_testing = archive.get_testing_data(sim)
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem, archive_testing[0]['sim_label'])

11 11 mpg_central
11 11 mpg_seed
11 11 mpg_growth
11 11 mpg_neutrinos
11 11 mpg_curved
11 11 mpg_running
11 11 mpg_reio
